In [74]:
'''
We need to make this more efficient. The easiest way to do this is to work out how often the moons 
repeat, and mathematically come up with the answer instead of having to repeat millions of times.

Each orbit would have a given period of time it takes to go around the system. Therefore, it should be a matter
of working out when each period aligns again.

Unfortunately, I couldn't quite work out how to do this, and only have the program before which tries to 
display the patterns the planets repeat in. I wasn't able to use this at all yet though! I figured if I 
could mathematically generate a formula based on the orbits of each planet, I could simply work out when
all the equations are equal to get an answer.

Oh well!
    
'''
import copy
import numpy as np

In [158]:
class Moon:
    '''
    An object containing each moon
    '''
    def __init__(self, name, x, y, z):
        self.name = name
        self.pos = [[int(x), int(y), int(z)]]
        self.vel = [[0, 0, 0]]
        self.repeatPattern = None

    def getPositionAtTime(self, time):
        '''
        returns the moon's position (coordinates) at a given time
        '''
        try:
            return self.pos[time]
        except:
            return None
    
    def getVelocityAtTime(self, time):
        '''
        returns a moon's velocity at a given time.
        '''
        try:
            return self.vel[time]
        except:
            return None
        
    def initTimeStep(self, time_step):
        '''
        initialises the time step provided to be the same as the step before it initially
        '''
        if len(self.pos) <= time_step:
            newpos = copy.deepcopy(self.pos[time_step-1])
            self.pos.append(newpos)
        if len(self.vel) <= time_step:
            newvel = copy.deepcopy(self.vel[time_step-1])
            self.vel.append(newvel)
    
    def getPositionString(self, time_step):
        '''
        Returns the current position of this moon to a string like on the puzzle's page
        '''
        return "x={:3}, y={:3}, z={:3}".format(self.pos[time_step][0], self.pos[time_step][1], self.pos[time_step][2])
    
    def getVelocityString(self, time_step):
        '''
        Returns the current velocity of this moon to a string like on the puzzle's page
        '''
        return "x={:3}, y={:3}, z={:3}".format(self.vel[time_step][0], self.vel[time_step][1], self.vel[time_step][2])
    
    def applyVelocity(self, time_step):
        '''
        Once we calculate the velocity of this position, we need to apply it to the position to update this step
        '''
        self.pos[time_step][0] += self.vel[time_step][0]
        self.pos[time_step][1] += self.vel[time_step][1]
        self.pos[time_step][2] += self.vel[time_step][2]
    
    def getPotentialEnergy(self, time_step):
        '''
        Calculates the potential energy in the moon for a given time step
        '''
        pot = abs(self.pos[time_step][0]) + abs(self.pos[time_step][1]) + abs(self.pos[time_step][2])
        return pot 
    
    def getKineticEnergy(self, time_step):
        '''
        Calculate the kinetic energy in the moon for a given time step
        '''
        pot = abs(self.vel[time_step][0]) + abs(self.vel[time_step][1]) + abs(self.vel[time_step][2])
        return pot
        
class System:
    '''
    This will store the information for the entire planetary system
    '''
    def __init__(self, moon_names, scan):
        self.moons = list()
        for idx, pos in enumerate(scan):
            pos_list = pos.replace("<","").replace(">","").replace("x=","").replace("y=","").replace("z=","").split(",")
            self.moons.append(Moon(moon_names[idx], pos_list[0], pos_list[1], pos_list[2]))
        
    def calculateVelocity(self, time_step):
        '''
        This calculates the velocity for the given time step based upon the differences in position.
        I probably could do this more elegantly, but this works for now...
        '''
        for i in range(len(self.moons)):
            self.moons[i].initTimeStep(time_step)
            for j in range(i + 1, len(self.moons)):
                self.moons[j].initTimeStep(time_step)
                if self.moons[i].pos[time_step][0] > self.moons[j].pos[time_step][0]:
                    self.moons[i].vel[time_step][0] -= 1
                    self.moons[j].vel[time_step][0] += 1
                elif self.moons[i].pos[time_step][0] < self.moons[j].pos[time_step][0]:
                    self.moons[i].vel[time_step][0] += 1
                    self.moons[j].vel[time_step][0] -= 1
                    
                if self.moons[i].pos[time_step][1] > self.moons[j].pos[time_step][1]:
                    self.moons[i].vel[time_step][1] -= 1
                    self.moons[j].vel[time_step][1] += 1
                elif self.moons[i].pos[time_step][1] < self.moons[j].pos[time_step][1]:
                    self.moons[i].vel[time_step][1] += 1
                    self.moons[j].vel[time_step][1] -= 1
                    
                if self.moons[i].pos[time_step][2] > self.moons[j].pos[time_step][2]:
                    self.moons[i].vel[time_step][2] -= 1
                    self.moons[j].vel[time_step][2] += 1
                elif self.moons[i].pos[time_step][2] < self.moons[j].pos[time_step][2]:
                    self.moons[i].vel[time_step][2] += 1
                    self.moons[j].vel[time_step][2] -= 1
        
        for moon in self.moons:
            moon.applyVelocity(time_step)
                    
    def getPosition(self, time_step):
        '''
        This will print out the position of the moons at a given time step
        '''
        print("After {} steps:".format(time_step))
        for moon in self.moons:
            if len(moon.name) > 6:
                print("{}:\t pos=<{}>, \t vel=<{}>".format(moon.name, moon.getPositionString(time_step), moon.getVelocityString(time_step)))
            else:
                print("{}:\t\t pos=<{}>, \t vel=<{}>".format(moon.name, moon.getPositionString(time_step), moon.getVelocityString(time_step)))
                    
    def getEnergy(self, time_step, print_step):
        '''
        This will calculate the energy for a given time step and print out the 
        results only if this is a printing step
        '''
        if time_step % print_step == 0:
            print("Energy after {} steps:".format(time_step))
        total = 0
        for moon in self.moons:
            pot = moon.getPotentialEnergy(time_step)
            kin = moon.getKineticEnergy(time_step)
            moon_total = kin * pot
            total += moon_total
            
            if time_step % print_step == 0:
                pot_str = "{:3} + {:3} + {:3} = {:3}".format(moon.pos[time_step][0], moon.pos[time_step][1], moon.pos[time_step][2], pot)
                kin_str = "{:3} + {:3} + {:3} = {:3}".format(moon.vel[time_step][0], moon.vel[time_step][1], moon.vel[time_step][2], kin)
                total_str = "{:3} * {:3} = {:4}".format(pot, kin, moon_total)

                if len(moon.name) > 6:
                    print("{}:\t pos={};\t vel={};\t total={}".format(moon.name, pot_str, kin_str, total_str))
                else:
                    print("{}:\t\t pos={};\t vel={};\t total={}".format(moon.name, pot_str, kin_str, total_str))

        if time_step % print_step == 0:
            print("Sum of total energy: {}".format(total))
            
    def timeShift(self, num_steps, print_step):
        '''
        This is the main controlling function that will loop over a given range of steps and make calculations
        '''
        jupiter.getPosition(0)
        for i in range(1, num_steps):
            jupiter.calculateVelocity(i)
            if i % print_step == 0:
                jupiter.getPosition(i)
            jupiter.getEnergy(i, print_step)
            
    def checkMoonPattern(self, moon, i):
        # Search through the position list for this moon, and find any duplicates with the current position
        idx = [m for m, v in enumerate(moon.pos[:-1]) if v == moon.pos[i]]
        
        # If we found any, then we can start to look at velocity...
        if len(idx) > 0:                
            # Pull the velocities in from the list of found positions
            velocities = [moon.vel[m] for m in idx]
            non_repeated_velocities = list()
            
            # Make a new list of items only if their velocities match also. Store the index
            for p, v in enumerate(velocities):
                if v not in velocities[:p] and v not in velocities[p+1:]:
                    non_repeated_velocities.append(p)
                    
            # Remove any items that don't have duplicate velocities
            for p in reversed(non_repeated_velocities):
                velocities.pop(p)

            # These are the indices with both repeated positions and velocities!
            if len(velocities) > 0:
                print("{} repeats in the following pattern at index {}:".format(moon.name, i))
                print("--> {}".format(idx))
            
    def timeToRepeat(self):
        looper = True
        i = 0
        idx = 0
        repeatedMoons = 0
        repeated = list()
        while looper:
            i += 1
            self.calculateVelocity(i)

            # Moon 1
            self.checkMoonPattern(self.moons[0], i)
                    
            # Moon 2
            self.checkMoonPattern(self.moons[1], i)
            
            # Moon 3
            self.checkMoonPattern(self.moons[2], i)
                    
            # Moon 4
            self.checkMoonPattern(self.moons[3], i)
                
            # We know the solution is here for our first test, so lets
            # artificially stop here for now...
            if i > 2772:
                looper = False
                

In [159]:
'''
Test 1
'''
moon_list = ["Io", "Europa", "Ganymede", "Callisto"]

test_scan = '''<x=-1, y=0, z=2>
<x=2, y=-10, z=-7>
<x=4, y=-8, z=8>
<x=3, y=5, z=-1>'''

scan = test_scan.split("\n")
jupiter = System(moon_list, scan)

jupiter.timeToRepeat()

Callisto repeats in the following pattern at index 635:
--> [47, 288, 354, 569]
Ganymede repeats in the following pattern at index 789:
--> [106, 722]
Ganymede repeats in the following pattern at index 865:
--> [142, 249, 758]
Callisto repeats in the following pattern at index 876:
--> [47, 288, 354, 569, 635]
Callisto repeats in the following pattern at index 904:
--> [19, 46, 355, 568, 877]
Ganymede repeats in the following pattern at index 925:
--> [7, 623]
Callisto repeats in the following pattern at index 943:
--> [19, 46, 355, 568, 877, 904]
Ganymede repeats in the following pattern at index 961:
--> [186, 205, 727, 802, 821]
Callisto repeats in the following pattern at index 970:
--> [19, 46, 355, 568, 877, 904, 943]
Callisto repeats in the following pattern at index 971:
--> [47, 288, 354, 569, 635, 876]
Io repeats in the following pattern at index 973:
--> [43, 49, 874, 880, 967]
Io repeats in the following pattern at index 975:
--> [32, 51, 116, 350, 573, 807, 872, 891, 956]


Io repeats in the following pattern at index 1172:
--> [80, 164, 248, 675, 759, 843, 1004, 1088]
Callisto repeats in the following pattern at index 1172:
--> [80, 164, 248, 675, 759, 843, 1004, 1088]
Callisto repeats in the following pattern at index 1173:
--> [146, 249, 674, 777, 1070]
Io repeats in the following pattern at index 1176:
--> [168, 252, 336, 587, 671, 755, 1092]
Callisto repeats in the following pattern at index 1176:
--> [168, 252, 336, 587, 671, 755, 1092]
Ganymede repeats in the following pattern at index 1177:
--> [29, 169, 586, 636, 785, 978]
Callisto repeats in the following pattern at index 1182:
--> [137, 258, 665, 786, 1061]
Callisto repeats in the following pattern at index 1184:
--> [260, 261, 327, 596, 662, 663]
Callisto repeats in the following pattern at index 1185:
--> [260, 261, 327, 596, 662, 663, 1184]
Callisto repeats in the following pattern at index 1186:
--> [7, 76, 160, 262, 661, 763, 847, 916, 931, 1000, 1084]
Io repeats in the following pattern a

Callisto repeats in the following pattern at index 1388:
--> [128, 459, 464, 795, 1052, 1383]
Io repeats in the following pattern at index 1389:
--> [458, 465, 1382]
Callisto repeats in the following pattern at index 1389:
--> [458, 465, 1382]
Io repeats in the following pattern at index 1390:
--> [457, 466, 1381]
Ganymede repeats in the following pattern at index 1390:
--> [149, 158, 1381]
Callisto repeats in the following pattern at index 1390:
--> [457, 466, 1381]
Io repeats in the following pattern at index 1391:
--> [456, 467, 1380]
Callisto repeats in the following pattern at index 1391:
--> [317, 456, 467, 606, 1241, 1380]
Io repeats in the following pattern at index 1392:
--> [300, 384, 455, 468, 539, 623, 1224, 1308, 1379]
Ganymede repeats in the following pattern at index 1392:
--> [763, 1379]
Callisto repeats in the following pattern at index 1392:
--> [300, 353, 384, 455, 468, 539, 570, 623, 1224, 1277, 1308, 1379]
Io repeats in the following pattern at index 1393:
--> [454

Callisto repeats in the following pattern at index 1481:
--> [161, 366, 557, 762, 1085, 1290]
Io repeats in the following pattern at index 1482:
--> [365, 558, 1289]
Ganymede repeats in the following pattern at index 1482:
--> [673, 1289]
Callisto repeats in the following pattern at index 1482:
--> [365, 558, 1289]
Io repeats in the following pattern at index 1483:
--> [364, 559, 1288]
Ganymede repeats in the following pattern at index 1483:
--> [672, 709, 1288, 1325, 1446]
Callisto repeats in the following pattern at index 1483:
--> [364, 559, 1288]
Io repeats in the following pattern at index 1484:
--> [279, 363, 447, 476, 560, 644, 1203, 1287, 1371, 1400]
Callisto repeats in the following pattern at index 1484:
--> [279, 363, 447, 476, 560, 644, 1203, 1287, 1371, 1400]
Io repeats in the following pattern at index 1485:
--> [362, 561, 1286]
Callisto repeats in the following pattern at index 1485:
--> [362, 561, 1286]
Io repeats in the following pattern at index 1486:
--> [361, 562, 1

--> [270, 653, 1194]
Ganymede repeats in the following pattern at index 1577:
--> [186, 205, 727, 802, 821, 961, 1194, 1343, 1428]
Callisto repeats in the following pattern at index 1577:
--> [270, 653, 1194]
Io repeats in the following pattern at index 1578:
--> [269, 654, 1193]
Callisto repeats in the following pattern at index 1578:
--> [132, 269, 654, 791, 1056, 1193]
Io repeats in the following pattern at index 1579:
--> [268, 655, 1192]
Callisto repeats in the following pattern at index 1579:
--> [268, 655, 1192]
Io repeats in the following pattern at index 1580:
--> [68, 267, 398, 525, 656, 855, 992, 1191, 1322, 1449]
Callisto repeats in the following pattern at index 1580:
--> [68, 134, 267, 656, 789, 855, 992, 1058, 1191]
Io repeats in the following pattern at index 1581:
--> [135, 200, 266, 657, 723, 788, 1059, 1124, 1190]
Ganymede repeats in the following pattern at index 1581:
--> [69, 574, 685, 1190]
Callisto repeats in the following pattern at index 1581:
--> [266, 657, 1

...

Callisto repeats in the following pattern at index 2572:
--> [199, 724, 1123, 1648, 2047]
Io repeats in the following pattern at index 2573:
--> [198, 725, 1122, 1649, 2046]
Callisto repeats in the following pattern at index 2573:
--> [198, 329, 594, 725, 1122, 1253, 1518, 1649, 2046, 2177, 2442]
Io repeats in the following pattern at index 2574:
--> [197, 726, 1121, 1650, 2045]
Callisto repeats in the following pattern at index 2574:
--> [197, 726, 1121, 1650, 2045]
Io repeats in the following pattern at index 2575:
--> [196, 727, 1120, 1651, 2044]
Ganymede repeats in the following pattern at index 2575:
--> [37, 196, 429, 578, 718, 812, 1045, 1334, 1437, 1726, 1959, 2053, 2193, 2342]
Callisto repeats in the following pattern at index 2575:
--> [196, 727, 1120, 1651, 2044]
Io repeats in the following pattern at index 2576:
--> [195, 392, 531, 728, 1119, 1316, 1455, 1652, 2043, 2240, 2379]
Ganymede repeats in the following pattern at index 2576:
--> [195, 289, 326, 811, 970, 1185, 1586

Io repeats in the following pattern at index 2608:
--> [163, 760, 1087, 1684, 2011]
Callisto repeats in the following pattern at index 2608:
--> [163, 760, 1087, 1684, 2011]
Io repeats in the following pattern at index 2609:
--> [162, 761, 1086, 1685, 2010]
Callisto repeats in the following pattern at index 2609:
--> [162, 761, 1086, 1685, 2010]
Io repeats in the following pattern at index 2610:
--> [155, 161, 762, 768, 1079, 1085, 1686, 1692, 2003, 2009]
Callisto repeats in the following pattern at index 2610:
--> [161, 366, 557, 762, 1085, 1290, 1481, 1686, 2009, 2214, 2405]
Io repeats in the following pattern at index 2611:
--> [7, 76, 160, 763, 847, 916, 931, 1000, 1084, 1687, 1771, 1840, 1855, 1924, 2008]
Ganymede repeats in the following pattern at index 2611:
--> [160, 776, 1995]
Callisto repeats in the following pattern at index 2611:
--> [7, 76, 160, 262, 661, 763, 847, 916, 931, 1000, 1084, 1186, 1585, 1687, 1771, 1840, 1855, 1924, 2008, 2110, 2509]
Io repeats in the followin

--> [11, 72, 95, 828, 851, 912, 935, 996, 1019, 1752, 1775, 1836, 1859, 1920, 1943]
Io repeats in the following pattern at index 2677:
--> [94, 829, 1018, 1753, 1942]
Ganymede repeats in the following pattern at index 2677:
--> [94, 318, 381, 626, 689, 850, 913, 1137, 1326, 1445, 1634, 1858, 1921, 2082, 2145, 2390, 2453]
Callisto repeats in the following pattern at index 2677:
--> [94, 829, 1018, 1753, 1942]
Io repeats in the following pattern at index 2678:
--> [93, 830, 1017, 1754, 1941]
Ganymede repeats in the following pattern at index 2678:
--> [93, 364, 401, 867, 1138, 1175, 1596, 1633, 1904, 2370, 2407]
Callisto repeats in the following pattern at index 2678:
--> [93, 176, 747, 830, 1017, 1100, 1671, 1754, 1941, 2024, 2595]
Io repeats in the following pattern at index 2679:
--> [92, 831, 1016, 1755, 1940]
Ganymede repeats in the following pattern at index 2679:
--> [92, 523, 2248]
Callisto repeats in the following pattern at index 2679:
--> [92, 831, 1016, 1755, 1940]
Io repeats

Io repeats in the following pattern at index 2712:
--> [59, 143, 227, 696, 780, 864, 983, 1067, 1151, 1620, 1704, 1788, 1907, 1991, 2075, 2544, 2628]
Ganymede repeats in the following pattern at index 2712:
--> [59, 248, 1291, 1480, 2523]
Callisto repeats in the following pattern at index 2712:
--> [59, 143, 227, 696, 780, 864, 983, 1067, 1151, 1620, 1704, 1788, 1907, 1991, 2075, 2544, 2628]
Io repeats in the following pattern at index 2713:
--> [58, 865, 982, 1789, 1906]
Ganymede repeats in the following pattern at index 2713:
--> [58, 165, 473, 534, 697, 842, 1066, 1173, 1290, 1481, 1598, 1705, 1929, 2074, 2237, 2298, 2606]
Callisto repeats in the following pattern at index 2713:
--> [58, 454, 469, 865, 982, 1378, 1393, 1789, 1906, 2302, 2317]
Io repeats in the following pattern at index 2714:
--> [57, 866, 981, 1790, 1905]
Ganymede repeats in the following pattern at index 2714:
--> [57, 365, 1174, 1597, 2406]
Callisto repeats in the following pattern at index 2714:
--> [57, 866, 98

Io repeats in the following pattern at index 2740:
--> [31, 892, 955, 1816, 1879]
Ganymede repeats in the following pattern at index 2740:
--> [31, 1200, 1263, 1508, 1571]
Callisto repeats in the following pattern at index 2740:
--> [31, 892, 955, 1816, 1879]
Io repeats in the following pattern at index 2741:
--> [30, 893, 954, 1817, 1878]
Ganymede repeats in the following pattern at index 2741:
--> [30, 338, 1201, 1570, 2433]
Callisto repeats in the following pattern at index 2741:
--> [30, 426, 497, 893, 954, 1350, 1421, 1817, 1878, 2274, 2345]
Io repeats in the following pattern at index 2742:
--> [29, 894, 953, 1818, 1877]
Ganymede repeats in the following pattern at index 2742:
--> [29, 169, 586, 636, 785, 978, 1177, 1252, 1519, 1594, 1793, 1986, 2135, 2185, 2602]
Callisto repeats in the following pattern at index 2742:
--> [29, 894, 953, 1818, 1877]
Io repeats in the following pattern at index 2743:
--> [28, 55, 139, 784, 868, 895, 952, 979, 1063, 1708, 1792, 1819, 1876, 1903, 19